In [47]:
from scrapfly import ScrapflyClient, ScrapeConfig, ScrapflyError
scrapfly = ScrapflyClient(key="scp-live-01fc8534987042f4a2fe553c5cf6a2df", max_concurrency=2)
url = 'https://www.bestbuy.com/site/6587182.p?skuId=6587182'
listing_result = scrapfly.scrape(ScrapeConfig(url=url, render_js=True, country="US", asp=True, retry=False, rendering_wait=10000))
listing_html = listing_result.scrape_result['content']

In [58]:
import re
def extract_json_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    script_tags = soup.find_all('script')

    json_data_list = []

    for script in script_tags:
        script_content = script.string
        if script_content:
            # Use a regular expression to find JSON-like content
            json_pattern = re.compile(r'({.*})', re.DOTALL)
            matches = json_pattern.findall(script_content)
            for match in matches:
                try:
                    json_data = json.loads(match)
                    json_data_list.append(json_data)
                except json.JSONDecodeError:
                    pass  # Ignore invalid JSON

    return json_data_list

In [64]:
# Function to clean the JSON and remove specified keys
def clean_json(data):
    if isinstance(data, dict):
        cleaned_data = {}
        for key, value in data.items():
            # Remove '@' from keys and skip 'context' and 'reviews' keys
            new_key = key.replace('@', '')
            if new_key == 'context' or new_key == 'reviews':
                continue
            # Recursively clean nested dictionaries or lists
            cleaned_data[new_key] = clean_json(value)
        return cleaned_data
    elif isinstance(data, list):
        return [clean_json(item) for item in data]
    else:
        return data

In [65]:
import jsonpath_ng as jp
import os
from bs4 import BeautifulSoup
import json
from pprint import pprint


def extract_json_data(soup, script_id):
        try:
            script_tag = soup.find('script', {'id': script_id, 'type': 'application/json'})
            if script_tag and script_tag.string:
                json_content = script_tag.string
                data = json.loads(json_content)
                return data
            else:
                print(f"No script tag found with id '{script_id}' or script content is empty.")
                return None
        except (json.JSONDecodeError, TypeError) as e:
            print(f"Error parsing JSON data: {e}")
            return None
        

input_dir = r"C:\Users\SudheerRChinthala\circana\Srikanth\Data"
input_file = os.path.join(output_dir, "BestBuyListing.html")
with open(input_file, 'w', encoding='utf-8') as f:
    f.write(listing_html)
# with open(output_file, 'r', encoding='utf-8') as f:
#     f.read() 
    
json_data_list = extract_json_from_html(listing_html)
# json_data = extract_json_data(soup, '__NEXT_DATA__')

# if isinstance(json_data, dict):
#     json_string = json.dumps(json_data)
# else:
#     json_string = json_data

# data = json.loads(json_string)


In [73]:
data = clean_json(json_data_list[10])
# if isinstance(json_data, dict):
#     json_string = json.dumps(json_data)
# else:
#     json_string = json_data
json_string = json.dumps(data)

In [74]:
json_string

'{"type": "Product", "name": "Acer - Chromebook 311 - 11.6\\" HD Laptop - Intel Celeron N4500 - 4GB Memory - 64GB eMMC - Star Black", "url": "https://www.bestbuy.com/site/acer-chromebook-311-11-6-hd-laptop-intel-celeron-n4500-4gb-memory-64gb-emmc-star-black/6587182.p?skuId=6587182", "description": "Shop Acer Chromebook 311 11.6\\" HD Laptop Intel Celeron N4500 4GB Memory 64GB eMMC Star Black at Best Buy. Find low everyday prices and buy online for delivery or in-store pick-up. Price Match Guarantee.", "sku": "6587182", "gtin13": "0195133259316", "model": "CBOA311-1H-C90F", "width": {"type": "http://schema.org/QuantitativeValue", "unitCode": "INH", "value": "7.83"}, "weight": {"type": "http://schema.org/QuantitativeValue", "unitCode": "LBR", "value": "2.65"}, "color": "Star Black", "brand": {"type": "Brand", "name": "Acer"}, "aggregateRating": {"type": "AggregateRating", "ratingValue": "4.5", "reviewCount": "20"}, "offers": {"type": "Offer", "priceCurrency": "USD", "price": "119", "avai

In [227]:
def flatten_json(nested_json, parent_key=''):
    items = {}
    for key, value in nested_json.items():
        if isinstance(value, dict):
            # Recursively flatten dictionaries
            items.update(flatten_json(value, parent_key))
        elif isinstance(value, list):
            # Handle lists, specifically flattening those with 'name' and 'value' structure
            for sub_item in value:
                if isinstance(sub_item, dict) and 'name' in sub_item and 'value' in sub_item:
                    # Use 'name' as key and 'value' as the corresponding value
                    items[sub_item['name']] = sub_item['value']
        else:
            # For all other types, just add them to the result
            items[key] = value
    return items

In [228]:
jp_first = lambda query, data: jp.parse(query).find(data)[0].value
jp_all = lambda query, data: [match.value for match in jp.parse(query).find(data)]

In [229]:
prop_data = jp_first("$..data", data)

In [230]:
result = {
    "ProductName": prop_data["product"]["name"],
    "UPC": prop_data["product"]["upc"],
    "description": prop_data["product"]["shortDescription"],
    "shippingText": prop_data["product"]["fulfillmentLabel"][0]["shippingText"] if prop_data["product"]["fulfillmentLabel"] else None,
    "returnPolicyText": prop_data["product"]["returnPolicy"]["returnPolicyText"],
    "priceString": prop_data["product"]["priceInfo"]["currentPrice"]["priceString"],
    "productHighlights":prop_data["idml"]["productHighlights"],
    "specifications":prop_data["idml"]["specifications"],
    "cannonicalURL":prop_data["product"]["canonicalUrl"],
    "images":prop_data["product"]["imageInfo"]["allImages"]    
}

In [232]:
final_json = json.dumps(result, indent = 4)

In [233]:
try:
    if isinstance(final_json, str):
        final_json = json.loads(final_json) 
    # Now flatten the parsed JSON
    flattened_data = flatten_json(final_json)
    flat_json_final = json.dumps(flattened_data)
except json.JSONDecodeError as e:
    pprint("Invalid JSON:", e)
except Exception as e:
    print("Error:", e)

In [234]:
print(flat_json_final)

{"ProductName": "Acer Predator Helios Neo 14.5 inch QHD+ 120Hz Gaming Laptop Intel Core Ultra 7-155H AI PC NVIDIA GeForce RTX 4070 16GB DDR5 RAM 1TB SSD Obsidian Black (2024)", "UPC": "195133232043", "description": "<p>Introducing the epitome of portable gaming excellence \u2013 the AI powered Predator Helios Neo 14, meticulously crafted for individuals who demand uncompromising performance for gaming and creativity without sacrificing the luxury of portability. Video game realms appearas realistic as ever on this AI-ready gaming laptop with 14.5\" WQXGA IPS display powered by an NVIDIA\u00ae GeForce RTX\u2122 4070 laptop GPU and the Intel\u00ae Core\u2122 Ultra 7 processor 155H. The Predator laptops also feature Microsoft\u2019s Copilot in Windows 11 and a dedicatedCopilot key for quick access to AI-powered task assistance.</p>", "shippingText": "Free shipping", "returnPolicyText": "Free 30-day returns", "priceString": "$1,599.00", "Screen size": "14.5 in", "Processor": "Intel Core Ul